In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Video Introduction
0,Male,32,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,...,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,Masters,I like playing sports,I give consent,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,Masters,No,I give consent,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,Female,32,Mumbai,,A supportive family & dependable friends,"Born and raised in Mumbai, though my roots are...",Never married,No,Between 30-35 lakhs,I am a Manager in Technology Consulting domain,...,Masters,I am looking for a life partner to grow and bu...,I give consent,Hindu,I am not,It will not work for me,No,Friend recommended,,
357,Female,36,Bangalore,,"Having a supportive family, good health, and b...",Grew up in a protective and close-knit family ...,Never married,No,Between 50-70 Lakhs,Corporate Communications Professional and wann...,...,Undergraduate,"I am ambitious, driven and career-oriented who...",I give consent,Hindu,I am not sure,It will not work for me,I am not sure,Other,,
358,Female,35,Delhi,,Job that have,Happy to have frnds,Never married,No,Under 12 Lakhs,Consulting services,...,High School,Nothing,I give consent,Hindu,Yes,It will not work for me,I am not sure,Instagram,,
359,Female,32,Bengaluru,,"Privilege, to me, means feeling financially s...","Growing up, there was a strong emphasis on ach...",Never married,No,Between 12-15 Lakhs,"I’m a data science consultant, but 

In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Video Introduction
2,Male,32,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,...,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,Masters,I like playing sports,I give consent,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,Masters,No,I give consent,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Female,32,Mumbai,,A supportive family & dependable friends,"Born and raised in Mumbai, though my roots are...",Never married,No,Between 30-35 lakhs,I am a Manager in Technology Consulting domain,...,Masters,I am looking for a life partner to grow and bu...,I give consent,Hindu,I am not,It will not work for me,No,Friend recommended,,
359,Female,36,Bangalore,,"Having a supportive family, good health, and b...",Grew up in a protective and close-knit family ...,Never married,No,Between 50-70 Lakhs,Corporate Communications Professional and wann...,...,Undergraduate,"I am ambitious, driven and career-oriented who...",I give consent,Hindu,I am not sure,It will not work for me,I am not sure,Other,,
360,Female,35,Delhi,,Job that have,Happy to have frnds,Never married,No,Under 12 Lakhs,Consulting services,...,High School,Nothing,I give consent,Hindu,Yes,It will not work for me,I am not sure,Instagram,,
361,Female,32,Bengaluru,,"Privilege, to me, means feeling financially s...","Growing up, there was a strong emphasis on ach...",Never married,No,Between 12-15 Lakhs,"I’m a data science consultant, but 

In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
358,Female,32,Mumbai,Never married
359,Female,36,Bangalore,Never married
360,Female,35,Delhi,Never married
361,Female,32,Bengaluru,Never married


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,"Was married earlier, legally divorced",Widow/ Widower
2,0,1,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
5,0,0,0,1,0,0
6,1,0,0,0,0,0
...,...,...,...,...,...,...
358,1,0,0,0,0,0
359,1,0,0,0,0,0
360,1,0,0,0,0,0
361,1,0,0,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
358,Female,32,Mumbai,Never married
359,Female,36,Bangalore,Never married
360,Female,35,Delhi,Never married
361,Female,32,Bengaluru,Never married


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
358,Female,32,Mumbai,Never married
359,Female,36,Bangalore,Never married
360,Female,35,Delhi,Never married
361,Female,32,Bengaluru,Never married


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(361, 143)


In [17]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 109 252  11 316 108 272 165 342  45 266  25 313 331 320  41
  330 278 332 326 247 245 343 290 351  48 156 261  24 311  81  26 229 139
  181 309 358 256 288 354 333 123  51  54  97 118 122  30 110 128 295 262
   84 136  86 349  19 231 260  55 325 259 274  75 336 307 310 114  42 183
  287 355 265 308  52 200 353 100  32   1  18  50  73 105 174  27 327 185
    6  29  12 258 111  65 344 251 228 178 301 282 291 246 221  77 166 121
   96 345  66 296 244  56 348  83 201 196  36 169 237 135 106 328 340 160
  209  89 194  91 143 241 335 255 208 357 163 171 219  71 321 352 107 230
  184 271 112 104 264  21 238 314  53 119 147 142 211 249 205 173 263 214
   35 141 242  76  62  70 191  79 137  94  37  59 180 315 257   9 297 101
  188 346 152  61  23 324 179 293 276 117  57   8 131  60 279 126  88  43
  250  13 319 212 130 195 223 146 339 217 132  22 133  14  10 170  46 189
  199 236  33 299  95 134 234 213 124 216 269 273 292 305 192 127 341 154
  145 253  16  72 285 329 232  85 233 

In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 36 'Delhi ' 'Never married']


['Male' 36 'Delhi' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 45 'New Delhi ' 'Was married earlier']


['Male' 41 'Bengaluru' 'Never married']


['Male' 50 'Bangalore' 'Widow/ Widower']


['Male' 35 'Delhi' 'Never married']


['Male' 40 'Bangalore ' 'Never married']


['Male' 40 'delhi' 'Was married earlier']


['Male' 40 'Hyderabad/pune' 'Never married

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.91529088  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.23401251  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.91529088  1.         ...  0.          0.
   0.        ]
 ...
 [ 0.         -0.40433211  1.         ...  0.          0.
   0.        ]
 [ 0.         -0.91529088  1.         ...  0.          0.
   0.        ]
 [ 1.         -1.08561048  1.         ...  0.          0.
   0.        ]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']